# Imports

In [81]:
#!pip install -U scikit-learn

In [82]:
from Transformation import Transformation
import numpy as np
from sklearn.neighbors import NearestNeighbors
import networkx as nx
import torch
from numpy import mean

# Input

In [83]:
transformation = Transformation()

user_number_points = 120
number_neigh_tri = 20

# Create objects
stl_file_path = "3d_models/stl/Handle.stl"
mesh_data = transformation.stl_to_mesh(stl_file_path)
graph = transformation.mesh_to_graph(mesh_data)

transformation.print_graph_properties(graph, display_graph=False, display_labels=False)

Number of nodes: 5999
Number of edges: 17991


In [84]:
if len(graph._node)<20:
    raise Exception("Input mesh does not have enough vertices. (More than 20 is needed)")

In [85]:
print(len(graph._node))

5999


# Point Sampler

### DevConv

In [86]:
def relu(array):
    return np.maximum(array, 0)

def sigmoid(array):
    return 1 / (1 + np.exp(-array))

In [87]:
class DevConv():
    def __init__(self, graph, output_dimension):
        self.graph = graph
        self.list_node = list(graph._node)

        self.W_phi = np.random.random((output_dimension))      #change
        self.W_theta = np.array([0.1, 0.1, 0.1]).reshape(1, -1)  # change
    
    def forward(self, previous_inclusion_score, return_flatten=True):
        list_inc_score = np.zeros((len(self.list_node), len(self.W_phi)))                               #list of "output_dimension" for each "list_node" element
        for index_current_node, (current_node, dict_neigh) in enumerate(self.graph._adj.items()):       # for each node and its adjacency nodes
            neigh_nodes = np.array(list(dict_neigh.keys()))                                             # array of adjacency nodes
            diff = np.subtract(current_node, neigh_nodes)                                               # Compute the differences between current_node and all neighbor nodes   (x_i - x_j)
            neigh_distances = np.linalg.norm(self.W_theta * diff, axis=1)                               # Compute the norm for each vector difference  ||W_theta * (x_i - x_j)||
            list_inc_score[index_current_node] = self.W_phi * np.max(neigh_distances)                   # Add (W_phi * ||W_theta * (x_i - x_j)||) to the inclusion score list

        if len(previous_inclusion_score)==0:                            # return if no previous inclusion score
            if return_flatten:
                list_inc_score = list_inc_score.flatten()
            return list_inc_score
        
        if list_inc_score.shape[1]!=1:                                  # If inclusion score is not vector
            list_inc_score = np.mean(list_inc_score, axis=1)            # Mean the matrix for each node

        # array of array to array
        if len(list_inc_score.shape)==2:                 
            if list_inc_score.shape[1]==1:
                list_inc_score = list_inc_score.flatten()

        # Return the mean of previous and current inclusion score
        return np.mean(np.array([previous_inclusion_score, list_inc_score], dtype=np.float64), axis=0)
        

In [88]:
devconv = DevConv(graph, 1)
inclusion_score = devconv.forward(previous_inclusion_score=np.array([]))
inclusion_score = relu(inclusion_score)
print(inclusion_score)

devconv = DevConv(graph, 64)
inclusion_score = devconv.forward(previous_inclusion_score=inclusion_score)
inclusion_score = relu(inclusion_score)
print(inclusion_score)

devconv = DevConv(graph, 1)
inclusion_score = devconv.forward(previous_inclusion_score=inclusion_score)
inclusion_score = sigmoid(inclusion_score)
print(inclusion_score)
print(inclusion_score.shape)

[1.15980235 1.16047713 1.15980235 ... 1.07973693 1.56732614 1.07973693]


[0.8854888  0.88600398 0.88548879 ... 0.82436025 1.19662607 0.82436025]
[0.64265846 0.64273688 0.64265846 ... 0.63330091 0.6885073  0.63330091]
(5999,)


### Multinomial Sampling

In [89]:
normalized_inclusion_score = inclusion_score / np.sum(inclusion_score)  # normalize for multinomial sampling
normalized_inclusion_score = np.round(normalized_inclusion_score, 8)    # round to remove float imprecision

number_throws = 500     # small:more randomness    |   big:less randomness
mult_sampling = np.random.multinomial(number_throws, normalized_inclusion_score)
print(mult_sampling)

[0 0 0 ... 0 0 0]


In [90]:
target_number_point = min(len(graph._node), user_number_points)   # number of points for the simplification

index_k_nodes = np.argpartition(mult_sampling, -target_number_point)[-target_number_point:]
list_k_nodes = np.array(list(graph._node.keys()))[index_k_nodes]
list_k_nodes = [tuple(e) for e in list_k_nodes]
print(list_k_nodes)

[(80.66123, -8.760632, -3.9920223), (75.10067, 1.3272538, 81.2), (92.00067, 2.9401166, 2.0131571), (75.630905, -3.8609922, 3.1690943), (76.69538, -4.7321377, -12.330506), (75.0, 5.0095077, 82.76503), (76.56288, -5.0095077, 3.718093), (75.36291, -3.8609922, 2.9849024), (51.0, -4.871695, 97.328476), (80.27039, 8.798424, -3.454077), (75.16284, -2.6241417, 3.4342492), (78.45044, 8.447791, 86.513214), (91.64078, 3.5589852, 1.1643598), (75.62203, 6.0434117, 1.4785457), (91.27412, 3.5589852, -0.56063664), (81.340294, 8.798424, -2.5402958), (81.47674, -8.672658, -4.198109), (82.254074, 8.798424, -1.4703954), (82.519966, 8.798424, -1.0835235), (51.0, -7.9077644, 93.86099), (86.74012, 8.294318, 68.3), (78.8188, -8.722893, 87.81436), (75.25122, 6.0434117, 1.4098194), (39.0, -8.177119, 93.25188), (92.32734, 0.99710757, 83.94438), (75.38672, -2.6241417, 3.696378), (76.82328, 1.6554987, 98.547325), (78.688194, -8.672658, -5.808077), (39.0, -8.699333, 91.327255), (51.0, -8.774797, 90.665535), (81.178

# KNN

In [91]:
XYZ = [list(e) for e in list_k_nodes]
print(XYZ)

[[80.66123, -8.760632, -3.9920223], [75.10067, 1.3272538, 81.2], [92.00067, 2.9401166, 2.0131571], [75.630905, -3.8609922, 3.1690943], [76.69538, -4.7321377, -12.330506], [75.0, 5.0095077, 82.76503], [76.56288, -5.0095077, 3.718093], [75.36291, -3.8609922, 2.9849024], [51.0, -4.871695, 97.328476], [80.27039, 8.798424, -3.454077], [75.16284, -2.6241417, 3.4342492], [78.45044, 8.447791, 86.513214], [91.64078, 3.5589852, 1.1643598], [75.62203, 6.0434117, 1.4785457], [91.27412, 3.5589852, -0.56063664], [81.340294, 8.798424, -2.5402958], [81.47674, -8.672658, -4.198109], [82.254074, 8.798424, -1.4703954], [82.519966, 8.798424, -1.0835235], [51.0, -7.9077644, 93.86099], [86.74012, 8.294318, 68.3], [78.8188, -8.722893, 87.81436], [75.25122, 6.0434117, 1.4098194], [39.0, -8.177119, 93.25188], [92.32734, 0.99710757, 83.94438], [75.38672, -2.6241417, 3.696378], [76.82328, 1.6554987, 98.547325], [78.688194, -8.672658, -5.808077], [39.0, -8.699333, 91.327255], [51.0, -8.774797, 90.665535], [81.178

In [92]:
def connect_extended_graph(XYZ, number_neigh=10):
    # Create the nearest neighbors object
    _, indices = NearestNeighbors(n_neighbors=number_neigh).fit(XYZ).kneighbors(XYZ)
    # Create the graph from the nearest points
    extended_graph = nx.Graph()
    for index_poly, poly in enumerate(indices):
        for index_current_node in range(len(poly)):
            current_node = tuple(XYZ[poly[index_current_node]])
            for index_other_node in range(index_current_node+1, len(poly)):
                edge = current_node, tuple(XYZ[poly[index_other_node]])
                extended_graph.add_edge(*edge)
                # if attribute do not exists
                if len(extended_graph.nodes[current_node])==0:
                    extended_graph.nodes[current_node]['index_poly'] = set()
                extended_graph.nodes[current_node]['index_poly'].add(index_poly)
    return extended_graph

In [93]:
extended_graph = connect_extended_graph(XYZ)

In [94]:
transformation.print_graph_properties(graph=extended_graph, display_graph=False, display_labels=False)

Number of nodes: 120
Number of edges: 1172


# Edge Predictor

In [95]:
devconv = DevConv(extended_graph, 64)
inclusion_score = devconv.forward(previous_inclusion_score=np.array([]), return_flatten=False)
inclusion_score.shape

(120, 64)

In [96]:
"""
inclusion_score = [[f_1_1  , f_1_2  , ..., f_63_1  ],
                    ...,
                   [f_1_M-1, f_1_M-1, ..., f_63_M-1]]
M = number of points
64 = hidden dimensions
"""


f = np.mean(inclusion_score, axis=1)                            # Flatten the matrix of inclusion score 
wq = np.random.rand(64)
wk = np.random.rand(64)

In [97]:
wq_f = wq.reshape(-1, 1) * f            # Wq*f
wk_f = wk.reshape(-1, 1) * f            # Wq*f
S = np.exp(np.dot(wq_f.T, wk_f))        # e^((wq_f.T)*(wk_f))

nodes = list(extended_graph.nodes())                                                                                # list of nodes
node_indices = {node: index for index, node in enumerate(nodes)}                                                    # dict{node: index}
neighbors_indices = [[node_indices[neighbor] for neighbor in extended_graph.neighbors(node)] for node in nodes]     # List of list : [[neigh1 of node1, neigh2 of node1...], [neigh1 of node2, neigh2 of node2...]...]


for index_current_node, neighbors_index in enumerate(neighbors_indices):        # For each neighbors of the 'index_current_node' node
    sum_columns = np.sum(S[:, neighbors_index], axis=1)                         # Sum along the rows the values of the neighbors
    S[index_current_node] = S[index_current_node] / sum_columns                 # And divide the current node columns by the sume of the neighbors

### Sparse Attention

# Face Candidates

#### Inputs

In [98]:
adjacency = nx.adjacency_matrix(extended_graph)
# S = np.random.rand(target_number_point, target_number_point)
print(adjacency)
print(S)

  (0, 1)	1
  (0, 2)	1
  (0, 3)	1
  (0, 4)	1
  (0, 5)	1
  (0, 6)	1
  (0, 7)	1
  (0, 8)	1
  (0, 9)	1
  (0, 25)	1
  (0, 26)	1
  (0, 40)	1
  (0, 41)	1
  (0, 42)	1
  (0, 43)	1
  (0, 44)	1
  (0, 45)	1
  (0, 51)	1
  (0, 52)	1
  (0, 69)	1
  (0, 106)	1
  (0, 108)	1
  (0, 109)	1
  (1, 0)	1
  (1, 2)	1
  :	:
  (118, 80)	1
  (118, 81)	1
  (118, 112)	1
  (118, 113)	1
  (118, 114)	1
  (118, 115)	1
  (118, 116)	1
  (118, 117)	1
  (119, 17)	1
  (119, 18)	1
  (119, 46)	1
  (119, 47)	1
  (119, 48)	1
  (119, 49)	1
  (119, 72)	1
  (119, 79)	1
  (119, 95)	1
  (119, 96)	1
  (119, 97)	1
  (119, 98)	1
  (119, 99)	1
  (119, 100)	1
  (119, 101)	1
  (119, 103)	1
  (119, 105)	1
[[2.46089252e-02 2.84825728e-02 3.02584356e-02 ... 1.37048912e-04
  1.45879043e-04 2.82768850e-02]
 [2.88727242e+05 9.39470627e-03 1.05574258e-02 ... 2.55557293e-06
  2.80239141e-06 9.26624935e-03]
 [1.64681491e+05 1.07546560e-01 6.46035302e-03 ... 4.25988581e-07
  4.73426270e-07 5.52183752e-03]
 ...
 [4.02233944e+20 2.18053289e+20 1.583032

In [99]:
A_s = np.zeros((target_number_point,target_number_point))   # Init A_s to 0

A_s = np.matmul(np.matmul(S, adjacency.toarray()), S.T)     # A_s = S * A * S.T

A_s = A_s/A_s.max()                                         # Normalize

In [100]:
print(A_s)  # symmétrique
print(A_s.shape)

[[6.01568302e-151 9.99078142e-146 5.69845148e-146 ... 5.13574757e-087
  2.91490599e-087 8.02096703e-131]
 [9.99078142e-146 6.51503839e-146 5.47553494e-146 ... 1.34055430e-087
  7.60860508e-088 1.10458266e-128]
 [5.69845148e-146 5.47553494e-146 4.12667868e-146 ... 7.44791727e-088
  4.22722523e-088 6.29989954e-129]
 ...
 [5.13574757e-087 1.34055430e-087 7.44791727e-088 ... 2.57451622e-022
  1.46124948e-022 4.46504927e-066]
 [2.91490599e-087 7.60860508e-088 4.22722523e-088 ... 1.46124948e-022
  8.29378729e-023 2.53424594e-066]
 [8.02096703e-131 1.10458266e-128 6.29989954e-129 ... 4.46504927e-066
  2.53424594e-066 7.37993829e-110]]
(120, 120)


# Face Classifier

### TriConv

#### Inputs

In [101]:
triangles = list(nx.simple_cycles(extended_graph, length_bound=3))  # [triangle0, triangle1,...] | triangle0 = [node1,node2,node3] | node1 = (x ,y ,z)
# triangles = cycles
print(triangles[0])
print(np.array(triangles).shape)    #nb_triangle, 3 nodes, 3 dimensions par node

[(80.66123, -8.760632, -3.9920223), (80.64282, -8.798424, -3.1683044), (81.47674, -8.672658, -4.198109)]
(4836, 3, 3)


In [102]:
p_init = np.zeros((len(triangles)))

for index_triangle, triangle in enumerate(triangles):
    i = list(dict(extended_graph._node).keys()).index(triangle[0])
    j = list(dict(extended_graph._node).keys()).index(triangle[1])
    k = list(dict(extended_graph._node).keys()).index(triangle[2])
    p_init[index_triangle] = (A_s[i,j] + A_s[i,k] + A_s[j,k])/3
print(p_init)
print(len(p_init))

[7.05492261e-146 2.36199544e-145 1.99162108e-145 ... 2.18770007e-092
 1.17136659e-092 3.57633199e-096]
4836


#### Calculate barycenter

In [103]:
barycenters = list()

for _, triangle in enumerate(triangles):
    b_x = (triangle[0][0] + triangle[1][0] + triangle[2][0]) / 3
    b_y = (triangle[0][1] + triangle[1][1] + triangle[2][1]) / 3
    b_z = (triangle[0][2] + triangle[1][2] + triangle[2][2]) / 3
    barycenters.append([b_x, b_y, b_z])

print(barycenters[0])
print(len(barycenters))

[80.9269307454427, -8.743904113769531, -3.7861455281575522]
4836


#### KNN Tri

In [104]:
_, indices_neigh_tri = NearestNeighbors(n_neighbors=number_neigh_tri).fit(barycenters).kneighbors(barycenters)


Diff Barycenters

In [105]:
barycenters_array = np.array(barycenters)
indices_neigh_tri_array = np.array(indices_neigh_tri)

# Calculate differences between points
barycenters_diff = np.subtract(barycenters_array[indices_neigh_tri_array[:, 0]][:, np.newaxis], barycenters_array[indices_neigh_tri_array[:, 1:]])   #Inverser la différence des barycentres si nécéssaire

print(barycenters_diff.shape)

(4836, 19, 3)


#### calculate e norm matrix

In [106]:
diff_vectors = list()

for _, triangle in enumerate(triangles):
    e_ij = np.linalg.norm(np.array(triangle[0]) - np.array(triangle[1]))
    e_ik = np.linalg.norm(np.array(triangle[0]) - np.array(triangle[2]))
    e_jk = np.linalg.norm(np.array(triangle[1]) - np.array(triangle[2]))
    diff_vectors.append([e_ij, e_ik, e_jk])
print(diff_vectors)
print(len(diff_vectors))

[[0.8247898, 0.8457312, 1.3310635], [0.8247898, 2.683034, 3.2870634], [0.8247898, 2.9828136, 2.3407323], [0.8247898, 6.2398176, 5.5416207], [0.8247898, 6.903221, 6.2398157], [0.8247898, 6.98207, 6.43868], [0.8247898, 7.166901, 6.5499506], [0.8247898, 7.539072, 7.7516737], [0.8247898, 10.074078, 10.773251], [0.8247898, 13.31874, 13.946552], [0.8247898, 7.621621, 8.179105], [0.8247898, 13.012527, 13.598226], [0.8247898, 14.523993, 15.004049], [0.8247898, 12.580704, 13.085766], [0.8247898, 9.04128, 8.496892], [0.8247898, 7.969537, 7.3745346], [0.8247898, 8.598248, 7.9269953], [0.8247898, 13.198326, 12.929761], [0.8247898, 9.078776, 8.538047], [0.8247898, 12.754551, 12.983127], [0.8457312, 2.683034, 3.2199335], [0.8457312, 2.9828136, 2.8391001], [0.8457312, 6.2398176, 6.0843816], [0.8457312, 6.903221, 6.6801043], [0.8457312, 6.98207, 6.5925884], [0.8457312, 7.166901, 6.870846], [0.8457312, 7.539072, 6.7374096], [0.8457312, 10.074078, 10.223744], [0.8457312, 13.31874, 13.43372], [0.8457312,

#### Calculate r

In [107]:
max_diff_vectors = np.array(diff_vectors).max(axis=1)       # calculate t_n_max
min_diff_vectors = np.array(diff_vectors).min(axis=1)       # calculate t_n_min
max_diff_vectors.shape

(4836,)

In [108]:
max_diff_vectors_diff = np.subtract(max_diff_vectors[indices_neigh_tri_array[:, 0]][:, np.newaxis], max_diff_vectors[indices_neigh_tri_array[:, 1:]])   #Inverser la différence des barycentres si nécéssaire   # calculate t_n_max - t_m_max
print(max_diff_vectors_diff.shape)
min_diff_vectors_diff = np.subtract(min_diff_vectors[indices_neigh_tri_array[:, 0]][:, np.newaxis], min_diff_vectors[indices_neigh_tri_array[:, 1:]])   #Inverser la différence des barycentres si nécéssaire   # calculate t_n_min - t_m_min
print(min_diff_vectors_diff.shape)

(4836, 19)
(4836, 19)


In [109]:
r_matrix = np.zeros((len(triangles), number_neigh_tri-1, 5))


# Fill r_matrix with the corresponding indices from indices_neigh_tri
r_matrix[:, :, 0] = min_diff_vectors_diff
r_matrix[:, :, 1] = max_diff_vectors_diff
r_matrix[:, :, 2:5] = barycenters_diff

print(r_matrix.shape)  # nb_triangles, len(indices_neigh_tri), 5dim/triangles

# Here r_matrix[i,j] represent the relation between the triangles (n = i) and (m = indices_neigh_tri_array[i,j])

(4836, 19, 5)


#### Calculate f

In [110]:
# MLP * 3 
f_final = p_init    # TODO


final_scores = torch.nn.functional.softmax(torch.tensor(f_final))
final_scores = final_scores.numpy()
print(final_scores.sum())
print(final_scores.shape)
print(final_scores)

0.9999999999999799
(4836,)
[0.00020678 0.00020678 0.00020678 ... 0.00020678 0.00020678 0.00020678]


C:\Users\ahdel\AppData\Local\Temp\ipykernel_3912\1064855114.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  final_scores = torch.nn.functional.softmax(torch.tensor(f_final))


# Simplified Mesh

In [111]:
selected_triangles_indexes = np.argpartition(final_scores, -target_number_point)[-target_number_point:] 
selected_triangles = np.array(triangles)[selected_triangles_indexes]
print(selected_triangles.shape) # number triangles, number points, number dimensions(x,y,z)

(120, 3, 3)


In [112]:
def create_triangle_graph(XYZ):
    extended_graph = nx.Graph()
    for index_poly, poly in enumerate(XYZ):
        for index_current_node in range(len(poly)):
            current_node = tuple(poly[index_current_node])
            for index_other_node in range(index_current_node+1, len(poly)):
                edge = current_node, tuple(poly[index_other_node])
                extended_graph.add_edge(*edge)
                # if attribute do not exists
                if len(extended_graph.nodes[current_node])==0:
                    extended_graph.nodes[current_node]['index_triangle'] = set()
                extended_graph.nodes[current_node]['index_triangle'].add(index_poly)
                if len(extended_graph.nodes[tuple(poly[index_other_node])])==0:
                    extended_graph.nodes[tuple(poly[index_other_node])]['index_triangle'] = set()
                extended_graph.nodes[tuple(poly[index_other_node])]['index_triangle'].add(index_poly)
    return extended_graph

In [113]:
selected_triangles

array([[[92.00067  ,  2.9401166,  2.0131571],
        [92.41     , -1.8187612, 29.6      ],
        [75.07711  , -1.1623888, 29.6      ]],

       [[92.00067  ,  2.9401166,  2.0131571],
        [92.41     , -1.8187612, 29.6      ],
        [79.64253  ,  7.755995 , 29.6      ]],

       [[92.00067  ,  2.9401166,  2.0131571],
        [79.64253  ,  7.755995 , 29.6      ],
        [89.079704 ,  7.0402236, 55.4      ]],

       ...,

       [[81.17586  , -8.399635 , 42.5      ],
        [76.56288  , -5.0095077,  3.718093 ],
        [79.64253  ,  7.755995 , 29.6      ]],

       [[79.64253  ,  7.755995 , 29.6      ],
        [76.56288  , -5.0095077,  3.718093 ],
        [80.859886 ,  8.294318 , 42.5      ]],

       [[79.64253  ,  7.755995 , 29.6      ],
        [76.56288  , -5.0095077,  3.718093 ],
        [84.133    ,  8.793697 , 29.6      ]]], dtype=float32)

In [114]:
simplified_final_graph = create_triangle_graph(selected_triangles)
print(simplified_final_graph._node)

{(92.00067, 2.9401166, 2.0131571): {'index_triangle': {0, 1, 2, 4, 6, 7, 8, 9, 10, 11, 22}}, (92.41, -1.8187612, 29.6): {'index_triangle': {0, 1, 101, 106, 111, 115, 116}}, (75.07711, -1.1623888, 29.6): {'index_triangle': {0, 99, 6, 7, 8, 107, 109, 110, 111, 112, 113, 114}}, (79.64253, 7.755995, 29.6): {'index_triangle': {1, 2, 97, 4, 103, 106, 108, 113, 117, 118, 119}}, (89.079704, 7.0402236, 55.4): {'index_triangle': {2, 6, 104, 9, 11, 107, 108}}, (82.519966, 8.798424, -1.0835235): {'index_triangle': {32, 33, 34, 3, 35, 5, 14, 17, 19, 21, 23, 25, 26, 27, 28, 29, 30, 31}}, (81.340294, 8.798424, -2.5402958): {'index_triangle': {3, 56, 37, 43, 49, 50, 51, 52, 21, 53, 23, 54, 25, 55}}, (81.22033, 8.610001, 0.63058394): {'index_triangle': {33, 66, 3, 35, 72, 41, 78, 47, 79, 80, 81, 82, 52, 59, 28, 31}}, (81.17586, -8.399635, 42.5): {'index_triangle': {98, 4, 7, 10, 11, 109, 117}}, (91.64078, 3.5589852, 1.1643598): {'index_triangle': {5, 12, 13, 14, 15, 16, 17, 18, 19, 20}}, (82.254074, 8.

In [115]:
transformation.print_graph_properties(graph=simplified_final_graph, display_graph=False, display_labels=False)

Number of nodes: 37
Number of edges: 145


In [116]:
simplified_final_mesh = transformation.graph_to_mesh(simplified_final_graph)

transformation.mesh_to_display_vtk(mesh_data)
transformation.mesh_to_display_vtk(simplified_final_mesh)